In [66]:
!pip install gradio jinja2


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [67]:
import gradio as gr 
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import TextVectorization

In [68]:
model = tf.keras.models.load_model('toxicity.keras')

In [69]:
MAX_WORDS = 200000

In [70]:
# INITILIZATION VECTORIZER TO ENCODE COMMENTS INTO INT
vectorizer = TextVectorization(max_tokens=MAX_WORDS, 
                           output_sequence_length=1800,
                           output_mode='int')

# Encode text into int using vectorizer
vectorizer.adapt(data['comment_text'].values)
vectorized_text = vectorizer(data['comment_text'].values)

In [71]:
data = pd.read_csv('train.csv')

In [72]:
column_headers = data.columns.tolist()[2:]

In [73]:
# Create subsets of clean and toxic comments
toxic_comments = data[data[column_headers].sum(axis=1) > 0]
clean_comments = data[data[column_headers].sum(axis=1) == 0]

In [74]:
# Handle dataset imbalance
train_clean_sample = clean_comments.sample(16225, random_state=42)
# Combine into one dataset
df = pd.concat([train_clean_sample, toxic_comments])
#Shuffle to avoid order bias
df = df.sample(frac=1, random_state=42)

In [75]:
comments = df['comment_text']

In [76]:
def classify_message(input):
    vectorized_input = vectorizer(input)
    results = model.predict(np.expand_dims(vectorized_input,0))

    text = ''
    for index, col in enumerate(data.columns[2:]):
        text += f'{col}: {results[0][index] > 0.5} \n'
    return text

In [77]:
# Encode text into int using vectorizer
vectorizer.adapt(comments.values)
vectorized_text = vectorizer(comments.values)

In [78]:
interface = gr.Interface(fn = classify_message, 
                        inputs = gr.Textbox(lines=2, placeholder='Enter you message here...'),
                        outputs = 'text')

In [79]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7875
Running on public URL: https://56c872c00f9012fdf0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
